# Splitting the Data (Train/Test)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Load the processed data
df = pd.read_csv('../data/telco_churn_processed.csv')

# 2. Separate Features (X) and Target (y)
# X = All columns EXCEPT 'Churn'
# y = Only 'Churn'
X = df.drop('Churn', axis=1)
y = df['Churn']

# 3. Split the data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape:  {X_test.shape}")

Training Data Shape: (5616, 30)
Testing Data Shape:  (1405, 30)
